In [8]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Load the ring image
ring = cv2.imread('ring.png', cv2.IMREAD_UNCHANGED)

# Helper function to overlay the ring
def overlay_ring(image, ring, x, y, width, height):
    ring_resized = cv2.resize(ring, (width, height))
    for i in range(height):
        for j in range(width):
            if ring_resized.shape[2] == 4 and ring_resized[i, j, 3] > 0:  # Image with alpha channel
                if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                    alpha = ring_resized[i, j, 3] / 255.0
                    image[y + i, x + j] = (1 - alpha) * image[y + i, x + j] + alpha * ring_resized[i, j, :3]
            elif ring_resized.shape[2] == 3:  # Image without alpha channel
                if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                    image[y + i, x + j] = ring_resized[i, j]

# Start the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a mirrored view
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks on the frame (optional)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the coordinates of the middle of the index finger
            pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
            dip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP]

            # Calculate the midpoint for ring placement
            x = int(((pip.x + dip.x) / 2) * frame.shape[1])
            y = int(((pip.y + dip.y) / 2) * frame.shape[0])
             # Adjust the position of the ring slightly below the current position
            y_offset = 55  # Increase this value to move the ring further down
            y += y_offset

            # Overlay the ring at the midpoint
            ring_width = 50  # Adjust size as needed
            ring_height = 50
            overlay_ring(frame, ring, x - ring_width // 2, y - ring_height // 2, ring_width, ring_height)

    # Show the frame
    cv2.imshow("Virtual Try-On: Ring", frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Load the earring image
earring = cv2.imread('earring.png', cv2.IMREAD_UNCHANGED)

# Helper function to overlay the earring
def overlay_earring(image, earring, x, y, width, height):
    earring_resized = cv2.resize(earring, (width, height))
    for i in range(height):
        for j in range(width):
            if earring_resized.shape[2] == 4 and earring_resized[i, j, 3] > 0:  # Image with alpha channel
                if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                    alpha = earring_resized[i, j, 3] / 255.0
                    image[y + i, x + j] = (1 - alpha) * image[y + i, x + j] + alpha * earring_resized[i, j, :3]
            elif earring_resized.shape[2] == 3:  # Image without alpha channel
                if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                    image[y + i, x + j] = earring_resized[i, j]

# Start the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a mirrored view
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Face Mesh
    result = face_mesh.process(rgb_frame)

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            # Get the coordinates for the left and right earlobes
            left_ear_x = int(face_landmarks.landmark[234].x * frame.shape[1])
            left_ear_y = int(face_landmarks.landmark[234].y * frame.shape[0])
            
            right_ear_x = int(face_landmarks.landmark[454].x * frame.shape[1])
            right_ear_y = int(face_landmarks.landmark[454].y * frame.shape[0])

            # Add offset for the lobe placement
            lobe_offset = 30  # Adjust this value for fine-tuning
            left_ear_y += lobe_offset
            right_ear_y += lobe_offset

            # Overlay the earring on both ears
            earring_width = 50  # Adjust size as needed
            earring_height = 50
            overlay_earring(frame, earring, left_ear_x - earring_width // 2, left_ear_y - earring_height // 2, earring_width, earring_height)
            overlay_earring(frame, earring, right_ear_x - earring_width // 2, right_ear_y - earring_height // 2, earring_width, earring_height)

    # Show the frame
    cv2.imshow("Virtual Try-On: Earrings", frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
